---
layout: code-post
title: How does training work?
tags: [neural nets]
---

Two questions:
- Is SGD purely statistical? 
    - can we just ignore it and do some approximation of it 
    by randomly moving around in a way that mimics SGD? (given an initialization scheme)
    - Aka how correlated are the changes of weights at a given level? Same node vs different node
    - What is the distribution of the changes for a given weight? Does it change by level? Which
    levels are most volatile? Does the back level change the most randomly and the top level
    the most consistently?
- How can we visualize training? Consider ReLU functions since these are almost the identity.
    - Picture grid after first activation
    - Picture grid after first two hidden layers
    - After three layers???